# Features
- retrieval routing
- metadata filter on year


In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## 1. Embedding

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

c:\Users\wongx\miniconda3\envs\langchain-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2. Indexing

In [3]:
# Load data

import json
from langchain.schema import Document

datapath = '../data/data_info.txt'

with open(datapath, "r") as file:
    raw_data = file.read()

corpus = json.loads(raw_data)
corpus[0].keys()

dict_keys(['title', 'abstract', 'keywords', 'year', 'doi', 'authors', 'full text', 'pages', 'content'])

In [4]:
# Preprocess and split data
from langchain.text_splitter import RecursiveCharacterTextSplitter

Abstract_Store = []

for thesis in corpus:
    document = Document(
    page_content=thesis['abstract'],
    metadata={
        "title": thesis['title'],
        "year": thesis['year'],
    })
    Abstract_Store.append(document)

print(len(Abstract_Store))
print(Abstract_Store[0].metadata)

Content_Store = []

for thesis in corpus:
    document = Document(
    page_content=thesis['content'],
    metadata={
        "title": thesis['title'],
        "year": thesis['year'],
    })
    Content_Store.append(document)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=100)
splits = text_splitter.split_documents(Content_Store)
print(len(splits))

100
{'title': 'A Critical Survey on the use of Fuzzy Sets in Speech and Natural Language Processing', 'year': 2012}
4716


In [5]:
# Store the data
from langchain_community.vectorstores import Chroma

Abstract_Store = Chroma.from_documents(documents=Abstract_Store, embedding=embedder, collection_name='abstract')
Content_Store = Chroma.from_documents(documents=splits[:10], embedding=embedder, collection_name='content')

KeyboardInterrupt: 

# 3.Routing - logical routing

Routing reference [here](https://python.langchain.com/v0.1/docs/use_cases/query_analysis/techniques/routing/)

In [7]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from tools.custom_chat_model import RedPillChatModel

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["Abstract_Store", "Content_Store", "OTHER"] = Field(
        ...,
        description="Abstract_Store is a database with abstracts of papers in the natural language field, Content_Store is a database with the full text of papers in the natural language field. Given a user question choose which datasource would be most relevant for answering their question. For Summarization or more general use cases, route to Abstract_Store, only if asked on concepts or specific content route to Content_Store. Otherwise, if you encounter something wierd or not in the field of nlp, return OTHER",
    )

# LLM with function call 
llm = RedPillChatModel(model="gpt-4o", 
                 api_key=os.getenv("RED_PILL_API_KEY"),
                 temperature = 0)
routing_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | routing_llm

In [8]:
Summary = router.invoke({"question": "summarize advancements in the field natural language processing on the year 2020"})
print(Summary.datasource)
Content = router.invoke({"question": "Tell me about the transformer architecture in detail"})
print(Content.datasource)
Other = router.invoke({"question": "我想看你洗澡"})
print(Other.datasource)

2024-12-03 21:32:16 - INFO - Sending request to Red Pill AI: {'model': 'gpt-4o', 'messages': [{'role': 'system', 'content': 'You are an expert at routing a user question to the appropriate data source.'}, {'role': 'user', 'content': 'summarize advancements in the field natural language processing on the year 2020'}], 'temperature': 0, 'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant datasource.', 'parameters': {'properties': {'datasource': {'description': 'Abstract_Store is a database with abstracts of papers in the natural language field, Content_Store is a database with the full text of papers in the natural language field. Given a user question choose which datasource would be most relevant for answering their question. For Summarization or more general use cases, route to Abstract_Store, only if asked on concepts or specific content route to Content_Store. Otherwise, if you encounter something wierd or not in 

Abstract_Store


2024-12-03 21:32:23 - INFO - Sending request to Red Pill AI: {'model': 'gpt-4o', 'messages': [{'role': 'system', 'content': 'You are an expert at routing a user question to the appropriate data source.'}, {'role': 'user', 'content': '我想看你洗澡'}], 'temperature': 0, 'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant datasource.', 'parameters': {'properties': {'datasource': {'description': 'Abstract_Store is a database with abstracts of papers in the natural language field, Content_Store is a database with the full text of papers in the natural language field. Given a user question choose which datasource would be most relevant for answering their question. For Summarization or more general use cases, route to Abstract_Store, only if asked on concepts or specific content route to Content_Store. Otherwise, if you encounter something wierd or not in the field of nlp, return OTHER', 'enum': ['Abstract_Store', 'Content_Store

Content_Store
OTHER


# 4. Self Querying Retrieval

self querying retrieval reference [here](https://python.langchain.com/docs/how_to/self_query/) and [here](https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb)

In [9]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from tools.customllm import RedPillLLM


metadata_field_info = [
    AttributeInfo(
        name="title",
        description="The title of the thesis",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the thesis was published",
        type="integer",
    ),
    AttributeInfo(
        name="abstract",
        description="The abstract of the thesis",
        type="integer",
    ),
]
document_content_description = "Thesis in the natural language processing field"

llm = RedPillLLM(model="gpt-4o", 
                 api_key=os.getenv("RED_PILL_API_KEY"),
                 temperature = 0.5)

Abstract_Retriever = SelfQueryRetriever.from_llm(
    llm,
    Abstract_Store,
    document_content_description,
    metadata_field_info,
    verbose=True,
    enable_limit=True,
)

Content_Retriever = SelfQueryRetriever.from_llm(
    llm,
    Content_Store,
    document_content_description,
    metadata_field_info,
    verbose=True,
    enable_limit=True,
)

In [10]:
Abstract_Retriever.invoke({"question":"give me developments in natural language processing field in 2020"})

2024-12-03 21:32:29 - INFO - Generated Query: query='developments in natural language processing' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='year', value=2020) limit=None


[Document(metadata={'title': 'Challenges and opportunities for public health made possible by advances in natural language processing', 'year': 2020}, page_content='Natural language processing (NLP) is a subfield of artificial intelligence devoted to understanding and generation of language. The recent advances in NLP technologies are enabling rapid analysis of vast amounts of text, thereby creating opportunities for health research and evidence-informed decision making. The analysis and data extraction from scientific literature, technical reports, health records, social media, surveys, registries and other documents can support core public health functions including the enhancement of existing surveillance systems (e.g. through faster identification of diseases and risk factors/at-risk populations), disease prevention strategies (e.g. through more efficient evaluation of the safety and effectiveness of interventions) and health promotion efforts (e.g. by providing the ability to obta

In [11]:
Content_Retriever.invoke({"question":"Explain the concept: Task Decomposition"})

2024-12-03 21:32:33 - INFO - Generated Query: query='Task Decomposition' filter=None limit=None


[Document(metadata={'title': 'Multi-Task Learning in Natural Language Processing: An Overview', 'year': 2024}, page_content='61,73–75,78,80,89,105,111,113,115,120,134,145,146,151,158,159,159,160,163,165,169,\n170]. For example, to prevent large datasets from dominating training, Perera et al. [ 98] set the\nweightsas\nλt∝1\n|Dt|,\nwhere|Dt|denotes the size of the training dataset for task t. The weights can also be adjusted\ndynamically during the training process based on certain metrics. Through adjusting weights,\nwe can purposely emphasize different tasks in different training stages. For instance, since dy-\nnamically assigning smaller weights to more uncertain tasks usually leads to good performance\nfor MTL [ 19,62] assigns weights based on the homoscedasticity of training losses from different\nACMComput.Surv., Vol. 56,No. 12,Article 295.Publicationdate:July 2024.Multi-Task LearninginNatural Language Processing: AnOverview 295:11\ntasksas\nλt=1\n2σ2\nt,\nwhereσtmeasuresthevaria

# 5. Generation

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})]


In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

abstract_chain = (
    {"context": Abstract_Retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

content_chain = (
    {"context": Content_Retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def choose_route(result):
    if "abstract_store" in result.datasource.lower():
        return 'abstract_chain'
    elif "content_store" in result.datasource.lower():
        return 'content_chain'
    else:
        return 'The answer that you are looking for is not here :)'


from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [14]:
query = "Give me 10 advancements in natural language processing field in 2020, answer in point form."
answer = full_chain.invoke({"question": query})

2024-12-03 21:32:33 - INFO - Sending request to Red Pill AI: {'model': 'gpt-4o', 'messages': [{'role': 'system', 'content': 'You are an expert at routing a user question to the appropriate data source.'}, {'role': 'user', 'content': 'Give me 10 advancements in natural language processing field in 2020, answer in point form.'}], 'temperature': 0, 'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant datasource.', 'parameters': {'properties': {'datasource': {'description': 'Abstract_Store is a database with abstracts of papers in the natural language field, Content_Store is a database with the full text of papers in the natural language field. Given a user question choose which datasource would be most relevant for answering their question. For Summarization or more general use cases, route to Abstract_Store, only if asked on concepts or specific content route to Content_Store. Otherwise, if you encounter something wierd

In [15]:
from IPython.display import Markdown
if answer == 'abstract_chain':
    docs  = Abstract_Retriever.invoke(query)
    print('_____Querrying Abstract Store_____')
    print('Here are the documents retrieved:')
    for doc in docs:
        print(doc.metadata['title'], doc.metadata['year'])

    response = abstract_chain.invoke(query)
    display(Markdown(response))

elif answer == 'content_chain':
    docs  = Abstract_Retriever.invoke(query)
    print('_____Querrying Content Store_____')
    print('Here are the documents retrieved:')
    print(docs)
    print('\n')
    response = abstract_chain.invoke(query)
    print('_____The response from LLM_____')
    display(Markdown(response))

else:
    print(answer)

2024-12-03 21:32:39 - INFO - Generated Query: query='advancements in natural language processing' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='year', value=2020) limit=10


_____Querrying Abstract Store_____
Here are the documents retrieved:
Challenges and opportunities for public health made possible by advances in natural language processing 2020
Natural language processing (NLP) in management research: A literature review 2020
Transformers: State-of-the-Art Natural Language Processing 2020
A Constant Time Complexity Spam Detection Algorithm for Boosting Throughput on Rule-Based Filtering Systems 2020
Language Models are Few-Shot Learners 2020
Improving the Reliability of Deep Neural Networks in NLP: A Review 2020
Identifying the Machine Learning Techniques for Classification of Target Datasets 2020
LANGUAGE MODEL IS ALL YOU NEED: NATURAL LANGUAGE UNDERSTANDING AS QUESTION ANSWERING 2020
Searching Better Architectures for Neural Machine Translation 2020
A Stacking-based Ensemble Learning Method for Outlier Detection 2020


2024-12-03 21:32:42 - INFO - Generated Query: query='advancements in natural language processing' filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='year', value=2020) limit=10


Based on the provided context, here are advancements in the field of natural language processing (NLP) in 2020:

1. **Rapid Analysis of Texts**: Advances in NLP technologies have enabled the rapid analysis of vast amounts of text, facilitating health research and decision-making.

2. **Transformer Architectures**: The development of Transformer architectures has allowed for building higher-capacity models, significantly impacting various NLP tasks.

3. **Pretraining Models**: Pretraining techniques have been improved, allowing effective utilization of model capacity across a wide variety of tasks.

4. **Few-Shot Learning**: The scaling up of language models, such as GPT-3, has improved task-agnostic, few-shot performance, nearing competitiveness with state-of-the-art fine-tuning approaches.

5. **Spam Detection**: A constant time complexity spam detection algorithm was developed, enhancing throughput in rule-based filtering systems.

6. **Adversarial Robustness**: Research has focused on improving the reliability of deep neural networks against adversarial examples, a significant hurdle in deploying these models in safety-critical environments.

7. **Neural Architecture Search for NMT**: A gradient-based neural architecture search algorithm was proposed for neural machine translation, discovering architectures that outperform existing models like Transformers.

8. **Transfer Learning in NLU**: Mapping natural language understanding problems to question answering problems has shown significant improvements in low data regimes.

9. **Ensemble Learning for Outlier Detection**: A stacking-based ensemble learning method was proposed to improve outlier detection performance using multiple base learners and a meta-learner.

10. **Open-Source Transformer Library**: The release of an open-source library for Transformer architectures has made these advances accessible to the wider machine learning community.

In [16]:
query = "Explain transformers in detail"
answer = full_chain.invoke({"question": query})

2024-12-03 21:32:50 - INFO - Sending request to Red Pill AI: {'model': 'gpt-4o', 'messages': [{'role': 'system', 'content': 'You are an expert at routing a user question to the appropriate data source.'}, {'role': 'user', 'content': 'Explain transformers in detail'}], 'temperature': 0, 'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant datasource.', 'parameters': {'properties': {'datasource': {'description': 'Abstract_Store is a database with abstracts of papers in the natural language field, Content_Store is a database with the full text of papers in the natural language field. Given a user question choose which datasource would be most relevant for answering their question. For Summarization or more general use cases, route to Abstract_Store, only if asked on concepts or specific content route to Content_Store. Otherwise, if you encounter something wierd or not in the field of nlp, return OTHER', 'enum': ['Abstrac

In [17]:
from IPython.display import Markdown
if answer == 'abstract_chain':
    docs  = Abstract_Retriever.invoke(query)
    print('_____Querrying Abstract Store_____')
    print('Here are the documents retrieved:')
    for doc in docs:
        print(doc.metadata['title'], doc.metadata['year'])

    response = abstract_chain.invoke(query)
    display(Markdown(response))

elif answer == 'content_chain':
    docs  = Abstract_Retriever.invoke(query)
    print('_____Querrying Content Store_____')
    print('Here are the documents retrieved:')
    print(docs)
    print('\n')
    response = abstract_chain.invoke(query)
    print('_____The response from LLM_____')
    display(Markdown(response))

else:
    print(answer)

2024-12-03 21:32:56 - INFO - Generated Query: query='transformers' filter=None limit=None


_____Querrying Content Store_____
Here are the documents retrieved:
[Document(metadata={'title': 'Transformers: State-of-the-Art Natural Language Processing', 'year': 2020}, page_content='Recent progress in natural language processing has been driven by advances in both model architecture and model pretraining. Transformer architectures have facilitated building higher-capacity models and pretraining has made it possible to effectively utilize this capacity for a wide variety of tasks. Transformers is an open-source library with the goal of opening up these advances to the wider machine learning community. The library consists of carefully engineered state-of-the art Transformer architectures under a unified API. Backing this library is a curated collection of pretrained models made by and available for the community. Transformers is designed to be extensible by researchers, simple for practitioners, and fast and robust in industrial deployments. The library is available at https://git

2024-12-03 21:33:01 - INFO - Generated Query: query='transformers' filter=None limit=None


_____The response from LLM_____


Transformers are a type of neural network architecture that have become foundational in modern natural language processing (NLP). Introduced in the paper "Attention Is All You Need" in 2017, transformers rely entirely on attention mechanisms, dispensing with the recurrence and convolutions traditionally used in sequence transduction models like recurrent neural networks (RNNs) and convolutional neural networks (CNNs).

### Key Features of Transformers:

1. **Attention Mechanism**: At the core of the transformer architecture is the attention mechanism, which allows the model to weigh the importance of different words in a sentence when making predictions. This mechanism enables the model to focus on relevant parts of the input sequence, making it highly effective for tasks that require understanding context and relationships between words.

2. **Parallelization**: Unlike RNNs, which process input sequentially, transformers allow for parallel processing. This is because each word in the input sequence is processed independently, making transformers much more efficient to train on modern hardware, such as GPUs.

3. **Scalability**: Transformers are highly scalable and can be trained on large datasets to build models with high capacity. This scalability has been a key factor in the success of transformer-based models like BERT and GPT.

4. **Pretraining and Fine-tuning**: Transformers are often pretrained on large corpora of text to learn general language representations. These pretrained models can then be fine-tuned on specific tasks, such as question answering or sentiment analysis, with relatively little additional data and training time. For example, BERT (Bidirectional Encoder Representations from Transformers) is a transformer model that is pretrained to understand the context of words from both directions in a sentence, making it highly effective for a variety of NLP tasks.

5. **Unified API**: Libraries like the open-source Transformers library by Hugging Face provide a unified API for implementing transformer architectures. This library includes a collection of pretrained models that are available to the community, making it easier for researchers and practitioners to use state-of-the-art models in their work.

Transformers have set new benchmarks in various NLP tasks, such as machine translation, language inference, and question answering, by achieving superior performance compared to previous models. They have become a standard tool in the field, driving significant progress and innovation in NLP research and applications.

In [18]:
query = "我想看你洗澡"
answer = full_chain.invoke({"question": query})

2024-12-03 21:33:10 - INFO - Sending request to Red Pill AI: {'model': 'gpt-4o', 'messages': [{'role': 'system', 'content': 'You are an expert at routing a user question to the appropriate data source.'}, {'role': 'user', 'content': '我想看你洗澡'}], 'temperature': 0, 'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant datasource.', 'parameters': {'properties': {'datasource': {'description': 'Abstract_Store is a database with abstracts of papers in the natural language field, Content_Store is a database with the full text of papers in the natural language field. Given a user question choose which datasource would be most relevant for answering their question. For Summarization or more general use cases, route to Abstract_Store, only if asked on concepts or specific content route to Content_Store. Otherwise, if you encounter something wierd or not in the field of nlp, return OTHER', 'enum': ['Abstract_Store', 'Content_Store

In [19]:
from IPython.display import Markdown
if answer == 'abstract_chain':
    docs  = Abstract_Retriever.invoke(query)
    print('_____Querrying Abstract Store_____')
    print('Here are the documents retrieved:')
    for doc in docs:
        print(doc.metadata['title'], doc.metadata['year'])

    response = abstract_chain.invoke(query)
    display(Markdown(response))

elif answer == 'content_chain':
    docs  = Abstract_Retriever.invoke(query)
    print('_____Querrying Content Store_____')
    print('Here are the documents retrieved:')
    print(docs)
    print('\n')
    response = abstract_chain.invoke(query)
    print('_____The response from LLM_____')
    display(Markdown(response))

else:
    print(answer)

The answer that you are looking for is not here :)
